In [1]:
import analyzer
import time
import pandas as pd
%run analyzer.py mnist_nets/mnist_relu_3_10.txt mnist_images/img0.txt 0.0005

Academic license - for non-commercial use only
verified
analysis time:  0.477675199508667  seconds


In [2]:
class interval_object:
    """Interval object to save the results of a box pass through the network
    Logits are saved in logit_intervals dictionary (they are needed to compute slope and intercept in full
    zonotope analysis)
    ReLU of logits are saved in intervals dictionary
    For the naming convention checkout the naming file."""
    def __init__(self, xl, xu):
        self.xl_in = xl
        self.xu_in = xu
        self.numlayer = 0
        self.intervals = {
            'xl_0': xl,
            'xu_0': xu
        }
        self.logit_intervals = {}
        self.fitted = False

    def set_logit_bounds(self, layer_k, hl, hu):
        assert self.numlayer > 0
        assert (layer_k < self.numlayer) and (layer_k >= 0)
        self.logit_intervals['hl_' + str(layer_k)] = hl
        self.logit_intervals['hu_' + str(layer_k)] = hu

    def set_bounds(self, layer_k, xl, xu):
        assert self.numlayer > 0
        assert (layer_k <= self.numlayer) and (layer_k >= 0)
        if not self.fitted:
            self.intervals['xl_' + str(layer_k)] = xl
            self.intervals['xu_' + str(layer_k)] = xu
        else:
            #only allow to tighten bounds
            for i in range(len(xl)):
                if xl[i] > self.intervals['xl_' + str(layer_k)][i]:
                    self.intervals['xl_' + str(layer_k)][i] = xl[i]
                if xu[i] < self.intervals['xu_' + str(layer_k)][i]:
                    self.intervals['xu_' + str(layer_k)][i] = xu[i]

    def get_bounds(self, layer_k):
        assert self.fitted
        assert(layer_k <= self.numlayer) and (layer_k >= 0)
        return self.intervals['xl_'+str(layer_k)], self.intervals['xu_'+str(layer_k)]

    def get_logit_bounds(self, layer_k):
        assert self.fitted
        assert(layer_k < self.numlayer) and (layer_k >= 0)
        return self.logit_intervals['hl_'+str(layer_k)], self.logit_intervals['hu_'+str(layer_k)]

    def get_input_bounds(self):
        # equivalent to get_bounds(layer_k = 0)
        return self.xl_in, self.xu_in

    def fit(self, nn, layer_k = 0):
        '''Fits with box from layer_k as input'''
        num_layer = nn.numlayer
        self.numlayer = num_layer
        xl = self.xl_in
        xu = self.xu_in
        num_pixels = len(xl)
        assert (num_pixels == len(xu))
        assert layer_k <= num_layer

        # Box pass through network and save intervals in self.intervals dictionary
        for k in range(layer_k, num_layer):
            layer_type = nn.layertypes[k]
            weights = np.array(nn.weights[k])
            biases = np.array(nn.biases[k])
            xl_k, xu_k = self.intervals['xl_'+str(k)], self.intervals['xu_'+str(k)]

            # Peform affine transform to get logits bounds hl_k, hu_k for kth layer
            hl_k, hu_k = analyze_layer_box(weights, biases, 'Affine', xl_k, xu_k)
            self.set_logit_bounds(k, hl_k, hu_k)

            # Perform relu operation to get new interval bounds xl_new, xu_new
            if layer_type == 'ReLU':
                xl_new = deepcopy(hl_k)
                xu_new = deepcopy(hu_k)
                xl_new[xl_new < 0] = 0
                xu_new[xu_new < 0] = 0
                self.set_bounds(k+1, xl_new, xu_new)

        self.fitted = True
        return xl_new, xu_new

    def print(self):
        # print('====== Input ======')
        # print_lower_upper(xl, xu)

        for k in range(self.numlayer):
            print('===== Logits bounds layer ' + str(k) + ' ======')
            print_lower_upper(*self.get_logit_bounds(k))

            print('===== Interval bounds layer ' + str(k + 1) + ' =====')
            print_lower_upper(*self.get_bounds(k + 1))

In [3]:
def relu_split(model, layer_k, dim_m, h_lb, h_ub, x_lb, x_ub, h_km):
    assert h_lb <= h_ub
    assert x_lb <= x_ub
    assert x_lb >= 0
    
    x_km = model.addVar(vtype=GRB.CONTINUOUS, name="x_"+str(layer_k)+'_' +str(dim_m))
    
    if h_ub < 0: #Zero case
        C0 = model.addLConstr(x_km == 0)
        constraints = [C0]
        flag = 0

    elif h_lb >= 0:
        C0 = model.addLConstr(x_km == h_km)
        constraints = [C0]
        flag = 1

    else:
        assert x_lb >= 0
        assert x_ub >= 0
        assert x_ub <= h_ub
    
        slope = x_ub / (x_ub - h_lb)
        intercept = -slope * h_lb
        assert intercept > 0
        
        C0 = model.addLConstr(x_km >= 0)
        C1 = model.addLConstr(x_km >= h_km)
        C2 = model.addLConstr(x_km <= slope * h_km + intercept)
        constraints = [C0,C1,C2]
        flag = 2
        
    return x_km, flag, constraints

In [4]:
def check_ub_convergence(x_ub, xu_old, abs_tol, rel_tol):
    if (xu_old - x_ub < abs_tol):
        #print('abs ub')
        return True
    if x_ub < 1e-5 or xu_old < 1e-5:
        x_ub = 0
        #print('0 ub')
        return True
    
    if (xu_old - x_ub)/xu_old < rel_tol:
        #print('rel ub')
        return True
    return False

In [5]:
def check_lb_convergence(x_lb, xl_old, abs_tol, rel_tol):
    if x_lb == 0:
        return True
    if (x_lb - xl_old  < abs_tol):
        #print('abs lb')
        return True
    if (x_lb - xl_old )/x_lb < rel_tol:
        #print('rel lb')
        return True
    return False

In [6]:
def tighten_bounds(model, x_km, flag, xl_old, xu_old, h_lb, h_km, constraints, abs_tol = 0.005, rel_tol = 0.01):  
    #Stable RELU case
    if len(constraints) == 1:
        model.reset()
        model.setObjective(x_km, GRB.MAXIMIZE)
        model.optimize()
        x_ub = x_km.X
        
        if x_ub > 0:
            model.reset()
            model.setObjective(x_km, GRB.MINIMIZE)
            model.optimize()
            x_lb = x_km.X
        else:
            x_lb = 0
        return x_lb, x_ub
    
    #RELU-Triangle case
    assert len(constraints) == 3
    model.reset()
    converged = False
    while not converged:
        model.setObjective(x_km, GRB.MAXIMIZE)
        model.optimize()
        x_ub = x_km.X
        
        if x_ub < 1e-4:
            model.remove(constraints[0])
            model.remove(constraints[1])
            model.remove(constraints[2])
            model.addLConstr(x_km == 0)
            model.update()
            x_ub = 0
            return 0, 0  #In this case both lower and upper bound ==0
        
        else: #slope update
            new_slope = x_ub / (x_ub - h_lb)
            new_intercept = -new_slope * h_lb
            assert new_intercept > 0
            model.remove(constraints[2])
            constraints[2] = model.addLConstr(x_km <= new_slope * h_km + new_intercept)
            #constraints[2].setAttr('rhs', new_slope * h_km + new_intercept)
            model.update()
        
        #Convergence check
        converged = check_ub_convergence(x_ub, xu_old, abs_tol, rel_tol)
        xu_old = x_ub
    
    #Find lower bound
    model.reset()
    converged = False
    while not converged:
        model.setObjective(x_km, GRB.MINIMIZE)
        model.optimize()
        x_lb = x_km.X
        
        #Convergence check
        converged = check_lb_convergence(x_ub, xu_old, abs_tol, rel_tol)
        xu_old = x_ub
        
    return x_lb, x_ub
        

In [7]:
def optimize_layer_l(layer_k, nn, iobj, model, varis, label): 
    '''Optimizes layer_k '''
    # Inits
    relu_bin = {'zero':0, 'lin':0, 'unstable':0, 'aff':0}
    
    # Get weights, biases & layertype from nn
    layer_type = nn.layertypes[layer_k-1]
    weights = np.array(nn.weights[layer_k-1])
    biases = np.array(nn.biases[layer_k-1])
    
    # Input and output dimensions for the k-th layer
    in_dim  = weights.shape[1]
    out_dim = weights.shape[0]
    
    # Get interval bounds from intervall object to calculate slope and intercept
    hl, hu = iobj.get_logit_bounds(layer_k-1)
    xl, xu = iobj.get_bounds(layer_k)
    assert hl.size == out_dim
    assert xl.size == out_dim       # The x's are ReLU(h)
    
    for dim_m in range(out_dim):
        ws = weights[dim_m,:]  #one row of weights
        b  = biases[dim_m]

        # Perform affine transform:  x_kn --> h_km where n iterates over input and m over output dimension
        h_km = LinExpr(b)
        for dim_n in range(in_dim):
            h_km.add( ws[dim_n] * varis['x_'+str(layer_k-1)+'_'+str(dim_n)] )

        # Activation
        if layer_type == 'ReLU':
            x_km, flag, constraints = relu_split(model, layer_k, dim_m, 
                                      h_lb=hl[dim_m], h_ub=hu[dim_m], x_lb=xl[dim_m], x_ub=xu[dim_m], 
                                      h_km=h_km)
            if flag == 0:
                relu_bin['zero'] +=1
            elif flag == 1:
                relu_bin['lin'] +=1
            elif flag == 2:
                relu_bin['unstable'] +=1
        else: #Affine case
            x_km = h_km
            relu_bin['aff']+=1
        
        # Add the variable x_km to variable dictionary
        varis['x_'+str(layer_k)+'_'+str(dim_m)] = x_km 
        
        #TIGHTEN STEP (Only for layers btw 1 & final)
        if xu[dim_m] > 0 and layer_k > 1:  #For first layer (1) this would just return box. 
            new_lb, new_ub = tighten_bounds(model, x_km, flag, xl[dim_m], xu[dim_m], hl[dim_m], h_km, constraints)
            
            assert new_lb >= xl[dim_m] - 1e-7
            assert new_ub <= xu[dim_m] + 1e-7
            
            #UPDATE Layer k
            iobj.intervals['xl_'+str(layer_k)][dim_m] = new_lb
            iobj.intervals['xu_'+str(layer_k)][dim_m] = new_ub
    
    #Final layer verification
    could_still_work = True
    if layer_k == (nn.numlayer):
        corr_label = varis['x_'+str(layer_k)+'_'+str(label)]
        
        for out_digit in range(10):
            if varis['x_'+str(layer_k)+'_'+str(out_digit)] is not corr_label:
                incorr_label = varis['x_' + str(layer_k) + '_' + str(out_digit)]
                
                model.reset()
                model.setObjective(corr_label - incorr_label, GRB.MINIMIZE)
                model.optimize()
                print(corr_label)
                print(incorr_label)

                if corr_label.X - incorr_label.X <= 0:
                    could_still_work = False
                    break
    
    return relu_bin, could_still_work

In [101]:
class Monitoring(): 
    ''' Monitoring remaining "allowed" runtime and changes in bounds. 
    Timing: Iterative timing bounds, i.e. check time consumption for last run
            and set current timer bound to last runtime. 
    Bounds: Check "importance" of each neuron, i.e. activation times weight. 
    Bounds Diff: Check change of bounds to stop optimizing bounds in case it 
                 of small optimization at each iteration. 
    '''
    MAX_TIME = 180.0 # 3min
    MIN_CHANGE = 1e-3
    
    def __init__(self): 
        self.time_last = None
        self.dt_last = self.MAX_TIME
        self.xl_last = None
        self.xu_last = None    
        
    def check(self, xl, xu):
        proceed = True
        dt_bounds = self.MAX_TIME
        # Check timing bound - Do not proceed if current runtime exceed  
        dt = self.MAX_TIME
        if self.time_last is None: 
            pass
        else: 
            dt = time.time() - self.time_last
            if dt > self.dt_last: 
                proceed = False
        dt_bounds = min(dt, self.MAX_TIME)
        # Check lower and upper bounds. 
        discard = []
        if self.xl_last is None or self.xu_last is None: 
            pass
        else: 
            dxl, dxu = self.xl_last - xl, self.xu_last - xu 
            lxl, lxu = self.xl_last, self.xu_last
            lxl[lxl == 0] = 1e-6 # avoid division error
            cxl = np.absolute(dxl/lxl) < self.MIN_CHANGE
            print(dxl)
            lxu[lxu == 0] = 1e-6 # avoid division error
            cxu = np.absolute(dxu/lxu) < self.MIN_CHANGE
            print(cxl)
        # Reset last parameters. 
        self.time_last = time.time()
        self.dt_last = dt
        self.xl_last = xl
        self.xu_last = xu
        return proceed, dt_bounds, discard
    

In [105]:
def aggressive_analyze(netname, epsilon, specname, iobj= None):
    verified = False
    #Import and parse the net
    with open(netname, 'r') as netfile:
        netstring = netfile.read()
    with open(specname, 'r') as specfile:
        specstring = specfile.read()
    nn = parse_net(netstring)
    x0_low, x0_high = parse_spec(specstring)
    LB_N0, UB_N0 = get_perturbed_image(x0_low,0)
    label, _ = analyze(nn,LB_N0,UB_N0,0)
    LB_N0, UB_N0 = get_perturbed_image(x0_low,epsilon)
    assert(LB_N0.shape == UB_N0.shape)
    
    #Set up variable, expression dictionarys & relu case counter
    varis = {}
    relu_bins = {}

    #First box pass
    if iobj is None:
        iobj = interval_object(LB_N0, UB_N0)
        iobj.fit(nn)
    if prove(*iobj.get_bounds(layer_k=nn.numlayer),label)[1]: return True, iobj, relu_bins, 0, label
    
    #set up gurobi model
    model = Model("net")
    model.setParam('OutputFlag', False)
    
    # set up monitoring. 
    monitoring = Monitoring()

    # Set up initial input variables & ranges for input image
    for dim_n in range(len(LB_N0)):
        x_0n  = model.addVar(LB_N0[dim_n], UB_N0[dim_n], name="x_0_"+str(dim_n))
        # Add the input variables to variable dict for later access under name x_0_n
        varis['x_0_'+str(dim_n)] = x_0n
        
    # Add 1st layer to model
    relu_bins[1], flag = optimize_layer_l(1, nn, iobj, model, varis, label)
    
    
    for layer_k in range(2, nn.numlayer):
        print('--', layer_k, '--')
        #print_lower_upper(iobj.get_bounds(layer_k)[0][:10],iobj.get_bounds(layer_k)[1][:10])
        #print('------------')
        
        relu_bins[layer_k], flag = optimize_layer_l(layer_k, nn, iobj, model, varis, label)
        iobj.fit(nn,layer_k)
        if prove(*iobj.get_bounds(layer_k=nn.numlayer),label)[1]: return True, iobj, relu_bins, layer_k, label
        
        # Iterative optimizing. 
        #model.setParam("TimeLimit", monitoring.MAX_TIME)
        for i in range(5): 
            print('----', layer_k, i, '--')
            proceed, dt_bounds, discard = monitoring.check(xl=iobj.get_bounds(layer_k)[0],
                                                           xu=iobj.get_bounds(layer_k)[1])
            if not proceed: break
            # Adapt model using monitoring results. 
            #model.setParam("TimeLimit", dt_bounds)
            # Optimization. 
            relu_bins[layer_k], flag = optimize_layer_l(layer_k, nn, iobj, model, varis, label)
            iobj.fit(nn,layer_k)
            if prove(*iobj.get_bounds(layer_k=nn.numlayer),label)[1]: return True, iobj, relu_bins, layer_k, label
        
        print('--', layer_k, ' finished--')
        #print_lower_upper(iobj.get_bounds(layer_k)[0][:10],iobj.get_bounds(layer_k)[1][:10])
        #print('------------')
        #print_lower_upper(*iobj.get_bounds(nn.numlayer))
    
    relu_bins[nn.numlayer], flag = optimize_layer_l(nn.numlayer, nn, iobj, model, varis, label)
    return flag, iobj, relu_bins, layer_k, label
    

In [106]:
netname = 'mnist_nets/mnist_relu_6_50.txt'
epsilon = 0.01
specname = 'mnist_images/img1.txt'

In [107]:
%%time
verf, iobj, relu_bins, layer_k, label = aggressive_analyze(netname, epsilon, specname)
print(verf, label, layer_k)
print(relu_bins)

-- 2 --
---- 2 0 --
---- 2 1 --
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True]


AssertionError: 

In [11]:
netname = 'mnist_nets/mnist_relu_6_200.txt'
epsilon = 0.01
specname = 'mnist_images/img0.txt'

In [ ]:
%%time
verf, iobj, relu_bins, layer_k, label = aggressive_analyze(netname, epsilon, specname)
print(verf, label, layer_k)
print(relu_bins)

-- 2 --
[0.00,	 0.93]
[0.00,	 0.01]
[0.00,	 0.28]
[0.00,	 1.17]
[0.00,	 1.42]
[0.00,	 1.33]
[0.00,	 0.67]
[0.00,	 0.00]
[0.00,	 1.38]
[0.00,	 1.93]
------------
-- 2 --
[0.00,	 0.12]
[0.00,	 0.01]
[0.00,	 0.01]
[0.00,	 0.29]
[0.00,	 0.51]
[0.00,	 0.55]
[0.00,	 0.02]
[0.00,	 0.00]
[0.05,	 0.66]
[0.05,	 0.83]
------------
[0.00,	 322.46]
[0.00,	 530.50]
[0.00,	 553.17]
[0.00,	 554.91]
[0.00,	 446.17]
[0.00,	 472.72]
[0.00,	 501.70]
[0.00,	 655.83]
[0.00,	 590.90]
[0.00,	 580.33]
-- 3 --
[0.00,	 2.65]
[0.00,	 0.86]
[0.00,	 2.13]
[0.00,	 0.30]
[0.00,	 0.00]
[0.00,	 0.97]
[0.00,	 0.95]
[0.00,	 0.00]
[0.00,	 0.92]
[0.00,	 1.30]
------------
-- 3 --
[0.17,	 1.76]
[0.00,	 0.08]
[0.00,	 1.25]
[0.00,	 0.02]
[0.00,	 0.00]
[0.00,	 0.20]
[0.00,	 0.19]
[0.00,	 0.00]
[0.00,	 0.15]
[0.00,	 0.69]
------------
[0.00,	 124.01]
[0.00,	 203.66]
[0.00,	 212.34]
[0.00,	 217.01]
[0.00,	 170.93]
[0.00,	 181.93]
[0.00,	 188.07]
[0.00,	 262.42]
[0.00,	 224.31]
[0.00,	 226.56]
-- 4 --
[0.00,	 3.14]
[0.00,	 2.07]


In [ ]:
netname = 'mnist_nets/mnist_relu_9_200.txt'
epsilon = 0.01
specname = 'mnist_images/img1.txt'